In [115]:
# The code was removed by Watson Studio for sharing.

# Course Applied Data Science Capstone
## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

For the current assignment it will be developed the following section
1. Creat a new Notebook
2. Scrape the Wikipwdia webseite https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
3. Create the dataframe shown in the asisgnment instructions

### 1. Crete a new Workbook

A new Notebook has been vreated in the Project Coursera Capstone in IBM Watson. The Notebook has been published in my GitHub account a a part of the repository Coursera Capstone.

### 2. Scrape the Wikipwdia webseite

In [1]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
print('The website to scrap is: ' , wikipedia_link)

The website to scrap is:  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


#### If necessary install the following modules

In [30]:
print('Installing module BeautifulSoup')
#!pip install beautifulsoup4
print('Installing module lxml')
#!pip install lxml
print('Installing module html5lib')
#!pip install html5lib
print('Installing module requests')
#!pip install request
print('Installing module geocoder')
#!pip install geocoder
print('Installing module Nominatim')
#!conda install -c conda-forge geopy --yes 

Installing module BeautifulSoup
Installing module lxml
Installing module html5lib
Installing module requests
Installing module geocoder
Installing module Nominatim


#### Import required modules

In [31]:
import requests
import csv
import pandas as pd
import numpy as np
import geocoder
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

Send a request to get the website and create a BeautifulSoup Object

In [104]:
r= requests.get(wikipedia_link)
soup = BeautifulSoup(r.text,'lxml')

Check the Soup Object "Website HTML code" to select the table to be scraped

In [105]:
#print(soup.prettify())

The table with the informatio to scrap has the class "wikitable sortable"

In [106]:
table = soup.find('table', class_='wikitable sortable')

Read the table and save the values in a DataFrame including the processing of values "Not assigned"

In [107]:
my_val = ['','','']
df2 = pd.DataFrame(columns=['postalcode', 'borough', 'neighborhood'])
for line in table.find_all('tr'):
    i = 0
    for elem in line.find_all('td'):
        if elem.text != "":
            my_val[i] = elem.text.strip()
            i = i+1
    if my_val[1] != "Not assigned":
        if my_val[2] == "Not assigned":
            df2 = df2.append(pd.Series([my_val[0],my_val[1],my_val[1]], index=df2.columns ), ignore_index=True)
        elif my_val[0] != '':
            df2 = df2.append(pd.Series([my_val[0],my_val[1],my_val[2]], index=df2.columns ), ignore_index=True)
        
            

In [108]:
df2.head()

,postalcode,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Join with "," the neighborhoods with the same postalcode

In [109]:
cols = df2.columns.tolist()
df2 = df2.groupby('postalcode').agg({'borough': 'first', 'neighborhood': ', '.join }).reset_index()
df2 = df2[cols]
df2.head()

,postalcode,borough,neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [149]:
df2.shape

(103, 3)

In [150]:
file = project.get_file("Geospatial_Coordinates.csv")
df = pd.read_csv(file)
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [153]:
df = df.rename(columns={'Postal Code': 'postalcode', 'Latitude' : 'latitude', 'Longitude' : 'longitud'})

In [157]:
df3 = pd.merge(df2, df, on='postalcode')

In [158]:
df3.head()

,postalcode,borough,neighborhood,latitude,longitud
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [162]:
df3[df3['postalcode'] == "M5G"]

,postalcode,borough,neighborhood,latitude,longitud
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
